In [1]:
import torchvision

teseset = torchvision.datasets.ImageNet(root='../data/torch_imagenet', split='val')

In [2]:
import torch
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms

In [3]:
if __name__ == "__main__":
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    transform = transforms.Compose(
        [transforms.Resize(256),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         normalize,
         ])

    test_set = torchvision.datasets.ImageNet(root='../data/torch_imagenet', transform=transform, split='val')
    test_loader = data.DataLoader(test_set, batch_size=100, shuffle=True, num_workers=4)

    model = torchvision.models.resnet152(pretrained=True).to(device)
    model.eval()

    correct_top1 = 0
    correct_top5 = 0
    total = 0

    with torch.no_grad():
        for idx, (images, labels) in enumerate(test_loader):

            images = images.to(device)      # [100, 3, 224, 224]
            labels = labels.to(device)      # [100]
            outputs = model(images)

            # ------------------------------------------------------------------------------
            # rank 1
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct_top1 += (pred == labels).sum().item()

            # ------------------------------------------------------------------------------
            # rank 5
            _, rank5 = outputs.topk(5, 1, True, True)
            rank5 = rank5.t()
            correct5 = rank5.eq(labels.view(1, -1).expand_as(rank5))

            # ------------------------------------------------------------------------------
            for k in range(6):
                correct_k = correct5[:k].reshape(-1).float().sum(0, keepdim=True)

            correct_top5 += correct_k.item()

            print("step : {} / {}".format(idx + 1, len(test_set)/int(labels.size(0))))
            print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))
            print("top-5 percentage :  {0:0.2f}%".format(correct_top5 / total * 100))
            
    print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))
    print("top-5 percentage :  {0:0.2f}%".format(correct_top5 / total * 100))

step : 1 / 500.0
top-1 percentage :  82.00%
top-5 percentage :  95.00%
step : 2 / 500.0
top-1 percentage :  80.00%
top-5 percentage :  93.50%
step : 3 / 500.0
top-1 percentage :  79.00%
top-5 percentage :  94.33%
step : 4 / 500.0
top-1 percentage :  79.00%
top-5 percentage :  94.50%
step : 5 / 500.0
top-1 percentage :  80.20%
top-5 percentage :  94.80%
step : 6 / 500.0
top-1 percentage :  79.50%
top-5 percentage :  94.33%
step : 7 / 500.0
top-1 percentage :  78.71%
top-5 percentage :  93.71%
step : 8 / 500.0
top-1 percentage :  78.88%
top-5 percentage :  93.50%
step : 9 / 500.0
top-1 percentage :  78.11%
top-5 percentage :  93.67%
step : 10 / 500.0
top-1 percentage :  77.80%
top-5 percentage :  93.70%
step : 11 / 500.0
top-1 percentage :  78.09%
top-5 percentage :  94.00%
step : 12 / 500.0
top-1 percentage :  78.42%
top-5 percentage :  93.83%
step : 13 / 500.0
top-1 percentage :  78.31%
top-5 percentage :  94.00%
step : 14 / 500.0
top-1 percentage :  78.29%
top-5 percentage :  94.14%
s